<a href="https://colab.research.google.com/github/catfoodlover/DATA608/blob/main/Module4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
#!pip install jupyter-dash
import pandas as pd
import numpy as np
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html


soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,boroname,health,steward,count(tree_id)' +\
        '&$group=spc_common,boroname,health,steward').replace(' ', '%20')


tree_counts = pd.read_json(soql_url)

#You have to drop missing values or the second graph wont load
tree_counts = tree_counts.dropna()

my_app = JupyterDash(__name__)

my_app.layout = html.Div(children=[
    html.H1(children = 'HEALTH OF NYC TREES'),
    dcc.Dropdown(
        id='dropdown-borough',
        options=[{'label': i, 'value': i} for i in ['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island']],
        value='Queens', multi=False, placeholder='Filter By Borough'
    ),
    html.Div(id='output-borough'),
    dcc.Dropdown(
        id='dropdown-steward',
        options=[{'label': i, 'value': i} for i in tree_counts['steward'].unique()],
        value='None', multi=False, placeholder='Filter by Steward'
    ),
    html.Div(id='output-steward'),
    ])

@my_app.callback(
        Output(component_id='output-borough', component_property='children'),
        [Input(component_id='dropdown-borough', component_property='value')]
        )

def boro_graph(input_data):
    df = tree_counts[tree_counts.boroname == input_data]
    
    return dcc.Graph(
            id='Tree Health in Borough',
            figure={
                    'data':[
              {'x':df['health'], 'type': 'histogram','name': 'Tree Health in Borough'}
          ],
          'layout':{
              'title':"Tree Health in Borough"
                  }
          }
              )

@my_app.callback(
        Output(component_id='output-steward', component_property='children'),
        [Input(component_id='dropdown-steward', component_property='value')]
        )

def steward_graph(input_data):
    df = tree_counts[tree_counts.steward == input_data]
    
    return dcc.Graph(
            id='Health by Steward',
            figure={
                    'data':[
              {'x':df['health'], 'type': 'histogram','name': 'Health by # Stewardship'}
          ],
          'layout':{
              'title':"Health by # Stewardship"
                  }
          }
              )


#if __name__ == '__main__':
my_app.run_server(mode='inline')
    #my_app.run_server(host='0.0.0.0', port=8050, debug=True)

<IPython.core.display.Javascript object>